<a href="https://colab.research.google.com/github/MedicalImageAnalysisTutorials/DeepLearning4All/blob/main/IA_DNN_ImageObjectDetection.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Introduction

**Draft version**
In this notebook, I will try to provide a practical tutorial for deep learning using simple examples. I will try to use simple implementation and avoid using built-in functions to give clear idea about the concept. You need basic programming knowledge. 

**The Object Detection Problem:**

**The Object Recognition Problem:**
                 .
I made the code flexible so one can try different approaches, datsaets, optimisers, loss functions based on if else statements. 

The general code structure:

    1. Read and pre-process the data
    2. Create the model, optimiser, loss function, and metrics
    3. Start the training loop
    4. Evaluate the final model




In [1]:
# TODO:
# complete the introduction
# add the introduction with the figures to this notebook https://github.com/idhamari/Deep-Learning-Coursera/blob/master/Convolutional%20Neural%20Networks/Week3/Car%20detection%20for%20Autonomous%20Driving/Autonomous_driving_application_Car_detection_v3a.ipynb
# define yolo model and its related functions 
# find dataset and use it 

import sys
# Setup 
doInstall =1
if doInstall:
    # !git clone https://github.com/JudasDie/deeplearning.ai/blob/master/Convolutional%20Neural%20Networks/week3/yolo_utils.py
    # from yolo_utils import read_classes, read_anchors, generate_colors, preprocess_image, draw_boxes, scale_boxes
    #!git clone https://github.com/allanzelener/yad2k.git
    !git  clone https://github.com/idhamari/YAD2K.git
    !pip install SimpleITK
    !pip install labelImg 
#   !git clone https://github.com/tensorflow/models.git 
#   !wget https://github.com/protocolbuffers/protobuf/releases/download/v3.18.0/protoc-3.18.0-linux-x86_64.zip
#   !unzip protoc-3.18.0-linux-x86_64.zip -d protoc-3.18.0
  #sys.path.append('/protoc-3.18.0/bin')
#   !export PATH="protoc-3.18.0/bin:$PATH"
#   !cd models/research
#   !protoc object_detection/protos/*.proto --python_out=.
#   !cd ../../cocoapi/PythonAPI
#   !git clone https://github.com/cocodataset/cocoapi.git
#   !make
#   !cp -r pycocotools <PATH_TO_TF>/TensorFlow/models/research/
    # !cd models/research
    # !cp object_detection/packages/tf2/setup.py .
    # !python -m pip install --use-feature=2020-resolver .
    # !mkdir workspace
    # !mkdir workspace/training_demo
    # #store all *.csv files and the respective TensorFlow *.record files, which contain the list of annotations for our dataset images.
    # !mkdir workspace/training_demo/annotations
    # #store exported versions of our trained model(s).
    # !mkdir workspace/training_demo/exported-models
    # #contain a sub-folder for each of training job. Each subfolder will contain the
    # # training pipeline configuration file *.config, as well as all files generated during the training and evaluation of our model.
    # !mkdir workspace/training_demo/models
    # !mkdir workspace/training_demo/pre-trained-models
    # #contains a copy of all the images in our dataset, as well as the respective *.xml files produced for each one, 
    # # once labelImg is used to annotate objects
    # !mkdir workspace/training_demo/images
    # !mkdir workspace/training_demo/images/train
    # !mkdir workspace/training_demo/images/test
import os, random, time, math, colorsys, imghdr, PIL

import numpy as np
import scipy.io
import scipy.misc
import pandas as pd

from skimage.transform import resize
import cv2 
import SimpleITK as sitk 
from PIL import Image, ImageDraw, ImageFont

import matplotlib.pyplot as plt

import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras import datasets, layers, models

from keras import backend as K
from keras.layers import Input, Lambda, Conv2D, BatchNormalization
from keras.models import load_model, Model


# Yolo
from yad2k.yad2k.models.keras_yolo import yolo_head, yolo_boxes_to_corners, preprocess_true_boxes, yolo_loss, yolo_body

%matplotlib inline

# to reproduce the same results given same input
np.random.seed(1)               
!ls


Cloning into 'YAD2K'...
remote: Enumerating objects: 243, done.
remote: Total 243 (delta 0), reused 0 (delta 0), pack-reused 243
Receiving objects: 100% (243/243), 2.35 MiB | 11.63 MiB/s, done.
Resolving deltas: 100% (106/106), done.
     |████████████████████████████████| 48.4 MB 4.7 kB/s 
     |████████████████████████████████| 247 kB 5.3 MB/s 
     |████████████████████████████████| 8.3 MB 29.2 MB/s 
     |████████████████████████████████| 317 kB 49.2 MB/s 
     |████████████████████████████████| 59.9 MB 35 kB/s 
  Created wheel for labelImg: filename=labelImg-1.8.5-py2.py3-none-any.whl size=258444 sha256=67856c82403a41b7365d85454aa56cfeeb20e176fb37fcbf0bf89eca2a71f48b
  Stored in directory: /root/.cache/pip/wheels/8a/fc/d0/41e6a87f4360fa551173280bd3c871e48711436e7873c6929c
Successfully built labelImg


ModuleNotFoundError: ignored

# Image Object Detection using CNN


## Reading and exploring the datasets

We are using Face Detection Data Set and Benchmark (FDDB) [dataset](http://vis-www.cs.umass.edu/fddb/README.txt) from university of Massachusetts.  


FDDB-folds contains files with names: FDDB-fold-xx.txt and FDDB-fold-xx-ellipseList.txt, where xx = {01, 02, ..., 10} represents the fold-index.

Each line in the "FDDB-fold-xx.txt" file specifies a path to an image in the above-mentioned data set. For instance, the entry "2002/07/19/big/img_130" corresponds to "originalPics/2002/07/19/big/img_130.jpg."

The corresponding annotations are included in the file "FDDB-fold-xx-ellipseList.txt" in the following 
format:



```
<image name i>
<number of faces in this image =im>
<face i1>
<face i2>
...
<face im>
```

Here, each face is denoted by:

            major_axis_radius minor_axis_radius angle center_x center_y 1


In [ ]:
datasetID         = 2  # 1:minst is selected by default, for cifar10 use 2
NNID              = 4  # 1:NN is by default, for DNN use 2,or 3, for 3D use 4  
number_of_classes = 10  # each datasets have 10 classes
showSamples = 1
doDownload = 0
# if you have large GPU memory you can combine the images to batches 
# for faster training.
# It is good to try different values
batch_size = 2 # you can try larger batch size e.g. 1024 * 6

# Facial dataset
# more info http://vis-www.cs.umass.edu/fddb/README.txt

if doDownload:
    !wget http://vis-www.cs.umass.edu/fddb/originalPics.tar.gz
    !tar zxvf  originalPics.tar.gz

    !wget http://vis-www.cs.umass.edu/fddb/FDDB-folds.tgz
    !tar zxvf FDDB-folds.tgz
    !mkdir originalPics
    !mv 2002 originalPics
    !mv 2003 originalPics




# (x_train, y_train), (x_test, y_test) = keras.datasets.mnist.load_data()
# class_names = range(10)

# if datasetID==2:
#     # cifar10 dataset
#     # The CIFAR10 dataset contains 60,000 color images in 10 classes, 
#     # with 6,000 images in each class.
#     # The dataset is divided into 50,000 training images and 10,000 testing images.
#     # The classes are mutually exclusive and there is no overlap between them.

#     (x_train, y_train), (x_test, y_test) = keras.datasets.cifar10.load_data()
#     class_names = ['airplane', 'automobile', 'bird', 'cat', 'deer',
#                'dog', 'frog', 'horse', 'ship', 'truck']
#     if NNID==4:
#         #TODO: fix this 
#         showSamples =0
        
#         x_train = x_train.reshape(-1,32*32*3) # (32 * 32 * 3)        
#         x_train = np.resize(x_train,(x_train.shape[0],15,15,15))        
#         x_train = x_train.reshape(-1,15,15,15)
#         x_test = x_test.reshape(-1,32*32*3) # (32 * 32 * 3)
#         x_test = np.resize(x_test,(x_test.shape[0],15,15,15))
#         x_test = x_test.reshape(-1,15,15,15)
#         x_train  =  x_train[..., np.newaxis] # np.reshape(x_train, (-1, h,w,1))
#         y_train  =  y_train[..., np.newaxis] # np.reshape(y_train, (-1, h,w,1))
#         x_test   =  x_test[..., np.newaxis]  # np.reshape(x_test,  (-1, h,w,1))
#         y_test   =  y_test[..., np.newaxis]  # np.reshape(y_test,  (-1, h,w,1))


#         print(x_train.shape)
#         print(x_test.shape)


# # get size 
# h = x_train.shape[1] # image height
# w = x_train.shape[2] # image width
# # check for rgb 
# try:
#     # number of channels
#     c =  x_train.shape[3]
# except:
#     # number of channels
#     c =  1
#     # if there is no number of channels, add 1
#     x_train  =  x_train[..., np.newaxis] # np.reshape(x_train, (-1, h,w,1))
#     y_train  =  y_train[..., np.newaxis] # np.reshape(y_train, (-1, h,w,1))
#     x_test   =  x_test[..., np.newaxis]  # np.reshape(x_test,  (-1, h,w,1))
#     y_test   =  y_test[..., np.newaxis]  # np.reshape(y_test,  (-1, h,w,1))


# # Reserve 10,000 samples for validation.
# x_val = x_train[-10000:]
# y_val = y_train[-10000:]
# x_train = x_train[:-10000]
# y_train = y_train[:-10000]

# number_of_pixels = h * w * c


# print("dataset shape   : ",x_train.shape)
# print("number of images: ",x_train.shape[0])
# print("image size      : ",x_train[0].shape)
# print("image data type : ",type(x_train[0][0][0][0]))
# print("image max  value: ",np.max(x_train[0]))
# print("image min  value: ",np.min(x_train[0]))
# if c==1:
#    print("gray or binary image (not color image)")
# elif c==3:
#    print("rgb color image (or probably non-color image represented with 3 channels)")


# # display sample images 
# if showSamples:
#     plt.figure(figsize=(10,10))
#     for i in range(25):
#         plt.subplot(5,5,i+1)
#         plt.xticks([])
#         plt.yticks([])
#         plt.grid(False)
#         #plt.imshow(x_train[i])
#         plt.imshow(cv2.cvtColor(x_train[i], cv2.COLOR_BGR2RGB))

#         # The CIFAR labels happen to be arrays, 
#         # which is why you need the extra index
#         if datasetID==1:
#             plt.xlabel(y_train[i])
#         elif datasetID==2:
#             plt.xlabel(class_names[y_train[i][0]])
#     plt.show()


# # normalisation
# x_train = np.array([ x/255.0 for x in x_train])
# x_val   = np.array([ x/255.0 for x in x_val])
# x_test  = np.array([ x/255.0 for x in x_test])
# #y_train = y_train.astype(np.float32)

# # for NN we need 1D 
# if NNID ==1:
#    x_train = np.reshape(x_train, (-1, number_of_pixels))
#    x_val   = np.reshape(x_val,  (-1, number_of_pixels))
#    x_test  = np.reshape(x_test , (-1, number_of_pixels))

# # Prepare the training dataset.
# print(x_train.shape,y_train.shape)
# train_dataset = tf.data.Dataset.from_tensor_slices((x_train, y_train))
# train_dataset = train_dataset.shuffle(buffer_size=1024).batch(batch_size)
# #train_dataset = tf.data.Dataset.from_tensor_slices((x_train, y_train)).batch(batch_size)

# # Prepare the validation dataset.
# val_dataset = tf.data.Dataset.from_tensor_slices((x_val, y_val))
# val_dataset = val_dataset.batch(batch_size)

# # Prepare the test dataset.
# tst_dataset = tf.data.Dataset.from_tensor_slices((x_test, y_test))
# tst_dataset = tst_dataset.batch(batch_size)

In [ ]:
import os 
from glob import glob
#(x_train, y_train), (x_test, y_test) = keras.datasets.mnist.load_data()
# class_names = range(10)
trnRatio = 0.90
tstRatio = 1 - trnRatio
img_fnms = sorted([y for x in os.walk("originalPics") for y in glob(os.path.join(x[0], '*.jpg'))])
lbls_fnms = sorted([y for x in os.walk("FDDB-folds") for y in glob(os.path.join(x[0], '*ellipseList.txt'))])
print((img_fnms[0:2]))
print(len(img_fnms))
print((lbls_fnms[0:2]))
print(len(lbls_fnms))
print("--------------------------")
lblInfo = []
numObj = 0
for fnm in lbls_fnms:
    f = open(fnm,'r')
    lines = f.readlines()
    imgLoc=[]
    for x in lines:
        if "img" in x: 
            imgPath = x.strip()
        elif "." in x:     
            imgLoc.append([float(y) for y in x.split()])
            if len(imgLoc) == numObj:
              lblInfo.append([imgPath,numObj,imgLoc,])                   
              imgLoc=[]
              numObj=0
        else:
            numObj = int(x)

print("---------------------------")
# annotations for 5171 faces
# 2845 images  ???
print(len(lblInfo))
s = 0 
for x in lblInfo:
#     print(x[0])
#     print(x[1])
      s = s  + x[1]
#     for i in range (x[1]):
#         print(x[2][i])
print(s)
# imgLst = [1             , 2               ]
# lblLst = [[1,3,[loc]]   , [1,3,[loc]]     ]

#(x_train, y_train), (x_test, y_test)
# class_names = range(10)

## Dataset augmentation

It is important to train the model on different variations of the dataset. It is also important to have large datset for training.

Using dataset augmentation helps to achieve both of the above goals. From one image, one can generate hundred thousands of images using image transformation.

The image transformation could be [spatial transform]() or point transform where we move the points of the image to new locations e.g. shifting, flipping, and/or rotating the imag. 

Another type of transformation is intensity transform or pixel transform where we change the color values of the pixels in the image e.g. invert the color, add more brightness or darkness. 



In [ ]:
# TODO
doAug = 1

def imagePixelTransforms(img):    
    images = []
    # let's make 3 simple transformations
    img1   = 1.0- img # invert color
    img2   = img +0.3 # more brightness
    img3   = img -0.3 # more darkness
    images = np.array([img1,img2,img3])
    images = [ img.reshape(img.shape) for img in images]

    # plt.figure() ;    plt.imshow(img)
    # plt.figure() ;    plt.imshow(img1)
    # plt.figure() ;    plt.imshow(img2)
    # plt.figure() ;    plt.imshow(img3)    
    return images

def imagePointTransforms(img):
    images = []
    # let's make 3 simple transformations
    # Perform the rotation
    center  = (img.shape[0] / 2, img.shape[1] / 2)
    sz      = (img.shape[1], img.shape[0])
    tMatrix = cv2.getRotationMatrix2D(center, 45, 1)
    img1 = cv2.warpAffine(img, tMatrix, sz)
    img1 = img1[...,np.newaxis] if img1.shape !=img.shape else img1
    tMatrix = cv2.getRotationMatrix2D(center, 90, 1)
    img2 = cv2.warpAffine(img, tMatrix, sz)
    img2 = img2[...,np.newaxis] if img2.shape !=img.shape else img2
    tMatrix = cv2.getRotationMatrix2D(center, 270, 1)
    img3 = cv2.warpAffine(img, tMatrix, sz)
    img3 = img3[...,np.newaxis] if img3.shape !=img.shape else img3

    images = np.array([img1,img2,img3])
    # plt.figure() ;    plt.imshow(img)
    # plt.figure() ;    plt.imshow(img1)
    # plt.figure() ;    plt.imshow(img2)
    # plt.figure() ;    plt.imshow(img3)    
    #plt.figure() ;    plt.imshow(cv2.cvtColor(img, cv2.COLOR_BGR2RGB))
    # plt.figure() ;    plt.imshow(cv2.cvtColor(img1, cv2.COLOR_BGR2RGB))
    return images


# define a function for sitk transform
def resample(img_array, transform):
    # Output image Origin, Spacing, Size, Direction are taken from the reference
    # image in this call to Resample
    image = sitk.GetImageFromArray(img_array)
    reference_image = image
    interpolator = sitk.sitkCosineWindowedSinc
    default_value = 100.0
    resampled_img = sitk.Resample(image, reference_image, transform,
                         interpolator, default_value)
    resampled_array = sitk.GetArrayFromImage(resampled_img)
    return resampled_array

def affine_rotate(transform, degrees):
    parameters = np.array(transform.GetParameters())
    new_transform = sitk.AffineTransform(transform)
    dimension =3 
    matrix = np.array(transform.GetMatrix()).reshape((dimension,dimension))
    radians = -np.pi * degrees / 180.
    rotation = np.array([[1  ,0,0], 
                         [0, np.cos(radians), -np.sin(radians)],
                         [0, np.sin(radians), np.cos(radians)]]
                        )
    new_matrix = np.dot(rotation, matrix)
    new_transform.SetMatrix(new_matrix.ravel())
    return new_transform


def imagePoint3DTransforms(img):
    #print("imagePoint3DTransforms")
    images = []
    # let's make 3 simple transformations
    # Perform the rotation
    # In SimpleITK resampling convention, the transformation maps points 
    # from the fixed image to the moving image,
    # so inverse of the transform is applied

    center = (img.shape[0] /2, img.shape[1] /2,img.shape[1] /2)
    rotation_around_center = sitk.AffineTransform(3)
    rotation_around_center.SetCenter(center)
    
    rotation_around_center = affine_rotate(rotation_around_center, -45)
    img1 = resample(img, rotation_around_center)

    rotation_around_center = affine_rotate(rotation_around_center, -90)
    img2 = resample(img, rotation_around_center)

    rotation_around_center = affine_rotate(rotation_around_center, -90)
    img3 = resample(img, rotation_around_center)

    images = np.array([img1,img2,img3])
    # plt.figure() ;    plt.imshow(img)
    # plt.figure() ;    plt.imshow(img1)
    # plt.figure() ;    plt.imshow(img2)
    # plt.figure() ;    plt.imshow(img3)    
    #plt.figure() ;    plt.imshow(cv2.cvtColor(img, cv2.COLOR_BGR2RGB))
    # plt.figure() ;    plt.imshow(cv2.cvtColor(img1, cv2.COLOR_BGR2RGB))
    return images

def doAugmentation(images,labels,batch_size):
    # input is an image or a batch e.g. list of images 
    # get numpy arrays from the tensor    
    images = images.numpy()
    labels = labels.numpy()
    # if 1d convert back to 2d
    #print(images.shape)
    rgb = 0 ; is3d = 0
    if len(images.shape) == 2:
       try: 
          img2d_shape = int(math.sqrt(images.shape[1])) # gray or binary image
          images =images.reshape(-1,img2d_shape,img2d_shape)
       except:
          try: 
            img2d_shape = int(math.sqrt(images.shape[1]/3)) # rgb image
            images =images.reshape(-1,img2d_shape,img2d_shape,3)
            rgb = 1  
          except:
            pass  
            # img3d_shape = int(math.sqrt(images.shape[1]/3)) # rgb image
            # images =images.reshape(-1,img2d_shape,img2d_shape,3)
            # is3d = 1  



    x_outputs = [] ; y_outputs = []
    i = 0
    for img in images:
        #print("-------------------------", i ,"--------------------")
        if NNID==4:
           img = img.squeeze() 
        # from each images we generate 6 images
        # 64 batch will generate 448
        x_outputs.extend([img])
        imgs1 = imagePoint3DTransforms(img)
        imgs2 = imagePixelTransforms(img)
        #if not rgb:
           #imgs1 = np.array( x[...,np.newaxis] for x in imgs1 if len(x.shape)<3) 
           #imgs2 = np.array( x[...,np.newaxis] for x in imgs2 if len(x.shape)<3)
        x_outputs.extend(imgs1) # 3 images
        x_outputs.extend(imgs2) # 3 images
        # print(img.shape)
        # print(imgs1[0].shape)
        # print(imgs2[0].shape)
        # assign the same label to all transformed images
        for j in range ( len(imgs1) +len(imgs2)+1):
            y_outputs.extend([labels[i]])

        i = i +1
    x_outputs = np.array(x_outputs)
    if NNID==4:
       x_outputs = np.array([x[...,np.newaxis] for x in x_outputs])
    y_outputs = np.array(y_outputs)

    if (not rgb) and (NNID==1):
       x_outputs = np.reshape(x_outputs, (-1,img2d_shape*img2d_shape,1))
    elif (rgb) and (NNID==1):
       x_outputs = np.reshape(x_outputs, (-1,img2d_shape*img2d_shape*3))   

    new_train_dataset = tf.data.Dataset.from_tensor_slices((x_outputs, y_outputs))
    new_train_dataset = new_train_dataset.shuffle(buffer_size=1024).batch(batch_size)

    return new_train_dataset

In [ ]:
# NN TensorFlow
def getNNModel(number_of_pixels,number_of_classes):
    inputs = keras.Input(shape=(number_of_pixels,), name="digits")
    x = layers.Dense(64, activation="relu", name="dense_1")(inputs)
    x = layers.Dense(64, activation="relu", name="dense_2")(x)
    outputs = layers.Dense(number_of_classes, name="predictions")(x)
    model = keras.Model(inputs=inputs, outputs=outputs)
    return model
print("NN model is defined ...")

In [ ]:
# TODO
# NN pytorch


## Define optimiser and loss function for NN and CNN

In [ ]:
# Instantiate an optimizer to train the model.

optimiserID = 1 # SGD by default for ADAM use 2 
optimizer = keras.optimizers.SGD(learning_rate=1e-3)
if optimiserID ==2:
   optimizer = keras.optimizers.Adam()#learning_rate=0.0001
# Instantiate a loss function.

lossFunctionID = 1 # SparseCategoricalCrossentropy by default for MSE use 2 
loss_fn = keras.losses.SparseCategoricalCrossentropy(from_logits=True)

# Prepare the metrics.
train_acc_metric = keras.metrics.SparseCategoricalAccuracy()
val_acc_metric   = keras.metrics.SparseCategoricalAccuracy()
tst_acc_metric   = keras.metrics.SparseCategoricalAccuracy()
if lossFunctionID==2:
   loss_fn = keras.losses.MeanSquaredError()
   # Prepare the metrics.
   train_acc_metric = keras.metrics.MeanSquaredError()
   val_acc_metric   = keras.metrics.MeanSquaredError()
   tst_acc_metric   = keras.metrics.MeanSquaredError()

elif  lossFunctionID==3:
   loss_fn = keras.losses.CategoricalCrossentropy()
   # Prepare the metrics.
   train_acc_metric = keras.metrics.CategoricalCrossentropy()
   val_acc_metric   = keras.metrics.CategoricalCrossentropy()
   tst_acc_metric   = keras.metrics.CategoricalCrossentropy()
print("optimiser, loss, and metrics are defined .... ")

In [ ]:
# TODO
# Pytorch



## Define required functions


In [ ]:

# define training parameters and file paths 

# model log files path
modelPath   = "./modelClassification.h5"
logFilePath = "./training_log.csv"
figPath     = "./training_log.png"

logFile = open(logFilePath,'w')
logFile.write("epoch \t trnLoss \t valLoss \t trnAcc \t valAcc \t time \n" )
logFile.close()
# Using optimised tensorflow functions provides more speed

@tf.function
def train_step(model,x, y):
    with tf.GradientTape() as tape:
        #print("get result")
        model_output = model(x, training=True)
        #print("get loss value ")
        #y = keras.utils.to_categorical(y)
        loss_value = loss_fn(y, model_output)
    grads = tape.gradient(loss_value, model.trainable_weights)
    optimizer.apply_gradients(zip(grads, model.trainable_weights))
    train_acc_metric.update_state(y, logits)
    return loss_value

@tf.function
def val_step(model,x, y):
    val_logits = model(x, training=False)
    #y = keras.utils.to_categorical(y)
    loss_value = loss_fn(y, val_logits)
    val_acc_metric.update_state(y, val_logits)
    return loss_value

# plotting function to monitor the curves
def iaPlotLoss(logPath,figPath=None):
    f = open(logPath,'r')
    lst = f.readlines()
    # first line is labels:
    labels = lst[0].split()[1:-2]
    x  = [ int(  ln.split()[0]) for ln in lst[1:]] # epoch
    y1 = [ float(ln.split()[1]) for ln in lst[1:]] # lossTrain
    y2 = [ float(ln.split()[2]) for ln in lst[1:]] # lossValidation
    y3 = [ float(ln.split()[3]) for ln in lst[1:]] # accTrain
    y4 = [ float(ln.split()[4]) for ln in lst[1:]] # accValidation
    #plotting    
    plt.clf()
    fig, ax = plt.subplots()    
    l1, = ax.plot(x, y1) ;     l2, = ax.plot(x, y2) ;
    l3, = ax.plot(x, y3) ;     l4, = ax.plot(x, y4) ;
    ax.legend((l1, l2,l3,l4), labels, loc='upper right', shadow=True)
    plt.xlabel('epoch')
    if figPath:
        plt.savefig(figPath, bbox_inches='tight')
    else:
        plt.show()
        plt.close()

#================================================
#                  YOLO
#================================================
def read_classes(classes_path):
    with open(classes_path) as f:
        class_names = f.readlines()
    class_names = [c.strip() for c in class_names]
    return class_names

def read_anchors(anchors_path):
    with open(anchors_path) as f:
        anchors = f.readline()
        anchors = [float(x) for x in anchors.split(',')]
        anchors = np.array(anchors).reshape(-1, 2)
    return anchors

def generate_colors(class_names):
    hsv_tuples = [(x / len(class_names), 1., 1.) for x in range(len(class_names))]
    colors = list(map(lambda x: colorsys.hsv_to_rgb(*x), hsv_tuples))
    colors = list(map(lambda x: (int(x[0] * 255), int(x[1] * 255), int(x[2] * 255)), colors))
    random.seed(10101)      # Fixed seed for consistent colors across runs.
    random.shuffle(colors)  # Shuffle colors to decorrelate adjacent classes.
    random.seed(None)       # Reset seed to default.
    return colors

def scale_boxes(boxes, image_shape):
    """ Scales the predicted boxes in order to be drawable on the image"""
    height = image_shape[0]
    width = image_shape[1]
    image_dims = K.stack([height, width, height, width])
    image_dims = K.reshape(image_dims, [1, 4])
    boxes = boxes * image_dims
    return boxes

def preprocess_image(img_path, model_image_size):
    image_type = imghdr.what(img_path)
    image = Image.open(img_path)
    resized_image = image.resize(tuple(reversed(model_image_size)), Image.BICUBIC)
    image_data = np.array(resized_image, dtype='float32')
    image_data /= 255.
    image_data = np.expand_dims(image_data, 0)  # Add batch dimension.
    return image, image_data

def draw_boxes(image, out_scores, out_boxes, out_classes, class_names, colors):   
    font = ImageFont.truetype(font='font/FiraMono-Medium.otf',size=np.floor(3e-2 * image.size[1] + 0.5).astype('int32'))
    thickness = (image.size[0] + image.size[1]) // 300

    for i, c in reversed(list(enumerate(out_classes))):
        predicted_class = class_names[c]
        box = out_boxes[i]
        score = out_scores[i]

        label = '{} {:.2f}'.format(predicted_class, score)

        draw = ImageDraw.Draw(image)
        label_size = draw.textsize(label, font)

        top, left, bottom, right = box
        top = max(0, np.floor(top + 0.5).astype('int32'))
        left = max(0, np.floor(left + 0.5).astype('int32'))
        bottom = min(image.size[1], np.floor(bottom + 0.5).astype('int32'))
        right = min(image.size[0], np.floor(right + 0.5).astype('int32'))
        print(label, (left, top), (right, bottom))

        if top - label_size[1] >= 0:
            text_origin = np.array([left, top - label_size[1]])
        else:
            text_origin = np.array([left, top + 1])

        # My kingdom for a good redistributable image drawing library.
        for i in range(thickness):
            draw.rectangle([left + i, top + i, right - i, bottom - i], outline=colors[c])
        draw.rectangle([tuple(text_origin), tuple(text_origin + label_size)], fill=colors[c])
        draw.text(text_origin, label, fill=(0, 0, 0), font=font)
        del draw


def yolo_filter_boxes(box_confidence, boxes, box_class_probs, threshold = .6):
    box_scores = box_confidence*box_class_probs
    box_classes = K.argmax(box_scores,-1)
    box_class_scores = K.max(box_scores,-1)
    filtering_mask = box_class_scores>threshold
    scores = tf.boolean_mask(box_class_scores,filtering_mask)
    boxes = tf.boolean_mask(boxes,filtering_mask)
    classes = tf.boolean_mask(box_classes,filtering_mask)
 
    return scores, boxes, classes      

# Intersection over Union
# check if predicted box is good iou>0.5
def iou(box1, box2):
    xi1 = max(box1[0],box2[0])
    yi1 = max(box1[1],box2[1])
    xi2 = min(box1[2],box2[2])
    yi2 = min(box1[3],box2[3])
    inter_area = (yi2-yi1)*(xi2-xi1)
    box1_area = (box1[3]-box1[1])*(box1[2]-box1[0])
    box2_area = (box2[3]-box2[1])*(box2[2]-box2[0])
    union_area = box1_area+box2_area-inter_area
    iou = inter_area/union_area
    return iou  

# find the best box from number of detcted boxes
def yolo_non_max_suppression(scores, boxes, classes, max_boxes = 10, iou_threshold = 0.5):
    max_boxes_tensor = K.variable(max_boxes, dtype='int32')
    K.get_session().run(tf.variables_initializer([max_boxes_tensor]))
    nms_indices = tf.image.non_max_suppression(boxes,scores,max_boxes,iou_threshold)
    scores = K.gather(scores,nms_indices)
    boxes = K.gather(boxes,nms_indices)
    classes = K.gather(classes,nms_indices)
    return scores, boxes, classes    

In [ ]:
##TODO pytorch

In [ ]:
if NNID==1:
    # Load the saved model 
    model = keras.models.load_model(modelPath, compile=False)

    start_time = time.time() 
    # Run a validation loop at the end of each epoch.
    for x_batch_tst, y_batch_tst in tst_dataset:
        output = model.predict(x_batch_tst)
        #y = keras.utils.to_categorical(y_batch_tst)
        tst_acc_metric.update_state(y_batch_tst, output)

    tst_acc = tst_acc_metric.result()

    # compute time required for each epoch
    end_time = time.time() - start_time

    print("test accuracy : %.4f \t time:  %.2f" % (  float(tst_acc), end_time))

In [ ]:
# TODO Pytorch


## Creating CNN model

In [ ]:
# Simple DNN
# just two conolution layers followed by dense layer
def getSimpleDNNModel(input_shape,number_of_pixels,number_of_classes):
    nF        = 16 # number of filters
    inputs = keras.Input(shape=input_shape, name="images") 
    # Create CNN model
    x11  = layers.Conv2D(nF, (3, 3), activation='relu', input_shape=input_shape) (inputs)
    x13  = layers.MaxPooling2D((2, 2)) (x11)
    x21  = layers.Conv2D(2*nF, (3, 3), activation='relu') (x13)
    x23  = layers.MaxPooling2D((2, 2))(x21)
    #dense layer for classification
    x31 = layers.Flatten()(x23)# convert from 3d to 1d
    outputs = layers.Dense(number_of_classes, name="predictions")(x31)
    model = keras.Model(inputs=inputs, outputs=outputs)
    return model

# this is a better model for CIFAR10
def getDNNModel(input_shape,number_of_pixels,number_of_classes):
    nF        = 64 # number of filters
    inputs = keras.Input(shape=input_shape, name="images") 
    # Create CNN model
    x11  = layers.Conv2D(nF, (3, 3), activation='relu', input_shape=input_shape) (inputs)
    x12  = layers.BatchNormalization()(x11)
    x13  = layers.MaxPooling2D((2, 2)) (x12)
    x14  = layers.Dropout(0.25)(x13)
    x21  = layers.Conv2D(2*nF, (3, 3), activation='relu') (x14)
    x22  = layers.BatchNormalization()(x21)
    x23  = layers.MaxPooling2D((2, 2))(x22)
    x24  = layers.Dropout(0.25)(x23)
    x31  = layers.Conv2D(2*nF, (3, 3), activation='relu')(x24)
    #dense layer for classification
    x41 = layers.Flatten()(x31)# convert from 3d to 1d
    #x7 = layers.Dense(2*nF, activation='relu')(x6)
    #x8 = layers.Dense(2*nF, activation='relu')(x7)
    x42  = layers.Dropout(0.50)(x41)
    outputs = layers.Dense(number_of_classes, name="predictions")(x42)
    model = keras.Model(inputs=inputs, outputs=outputs)
    return model

def getSimpleDNN3DModel(input_shape,number_of_pixels,number_of_classes):
    nF        = 16 # number of filters
    inputs = keras.Input(shape=input_shape, name="images") 
    # Create CNN model
    x11  = layers.Conv3D(nF, (3, 3, 3), activation='relu', input_shape=input_shape) (inputs)
    x13  = layers.MaxPooling3D((2, 2, 2)) (x11)
    x21  = layers.Conv3D(2*nF, (3, 3, 3), activation='relu') (x13)
    x23  = layers.MaxPooling3D((2, 2 ,2))(x21)
    #dense layer for classification
    x31 = layers.Flatten()(x23)# convert from 3d to 1d
    outputs = layers.Dense(number_of_classes, name="predictions")(x31)
    model = keras.Model(inputs=inputs, outputs=outputs)
    return model

#========================================
#              Yolo  
#========================================


def getYoloDNNModel:
    model = 0

    return model    


print("DNN model is defined ...")    


## Training CNN model

In [ ]:
# Same code as above 

epochs = 500 # number of iterations 

if NNID>=2:
    input_shape = [h,w,c] 

    if NNID==2:
       model = getSimpleDNNModel(input_shape, number_of_pixels,number_of_classes)
    elif NNID==3: # advanced 
       model = getDNNModel(input_shape, number_of_pixels,number_of_classes)
    elif NNID==4: # 3D 
       input_shape = [h,w,c,1] 
       model = getSimpleDNN3DModel(input_shape, number_of_pixels,number_of_classes)

    print("===================================================")
    print("               Training Loop           ")
    print("===================================================")
    total_time_start = time.time()
    # we loop number of iterations
    # for each iteration, we loop through all the training samples
    for epoch in range(epochs):
        #print("\nStart of epoch %d" % (epoch,))
        start_time = time.time()

        # Iterate over the batches of the dataset.
        for step, (x_batch_train, y_batch_train) in enumerate(train_dataset):
            #print(train_dataset.shape)
            #print(x_batch_train.shape,y_batch_train.shape)
            if doAug: 
                #do augmentation
                new_train_batch = doAugmentation(x_batch_train , y_batch_train , batch_size)                
                for stp, (new_x_batch_train, new_y_batch_train) in enumerate(new_train_batch):
                    #print(stp)
                    #print(new_train_batch.shape)
                    #print(new_x_batch_train.shape,new_y_batch_train.shape)
                    #model.summary()
                    loss_value = train_step(model,new_x_batch_train, new_y_batch_train)
                    train_acc = train_acc_metric.result()
                    train_acc_metric.reset_states()
                    print("   epoch:%d \t stp %d trnLoss: %.4f " % (epoch, stp, float(loss_value)))
            else:                    
                loss_value = train_step(model,x_batch_train, y_batch_train)                    
                train_acc = train_acc_metric.result()
                train_acc_metric.reset_states()

        # Run a validation loop at the end of each epoch.
        for x_batch_val, y_batch_val in val_dataset:
            val_loss_value = val_step(model,x_batch_val, y_batch_val)

        val_acc = val_acc_metric.result()
        val_acc_metric.reset_states()
        
        # compute time required for each epoch
        end_time = time.time() - start_time

        print("epoch:%d \t trnLoss: %.4f \t valLoss: %.4f \t trnAcc: %.4f \t valAcc: %.4f \t time:  %.2f" % (epoch, float(loss_value),float(val_loss_value), float(train_acc), float(val_acc), end_time))
        logFile = open(logFilePath,'a')
        logFile.write("%d \t %.4f \t  %.4f \t %.4f \t  %.4f \t  %.2f \n" % (epoch, float(loss_value),float(val_loss_value), float(train_acc), float(val_acc), end_time))
        logFile.close()
        if epoch % 5 ==0:
           # plot the result        
           iaPlotLoss(logFilePath)
           model.save(modelPath)      
    # save the final model
    model.save(modelPath)     

    # plot the result        
    iaPlotLoss(logFilePath)
    total_time_end = time.time() - total_time_start
    print("Training this dataset took ", total_time_end," seconds!") 
    print("Training this dataset took ", total_time_end/60.0," minutes!") 


## Evaluation

In [ ]:
if NNID==2:
    # Load the saved model 
    model = keras.models.load_model(modelPath, compile=False)

    start_time = time.time() 
    # Run a validation loop at the end of each epoch.
    for x_batch_tst, y_batch_tst in tst_dataset:
        output = model.predict(x_batch_tst)
        #y = keras.utils.to_categorical(y_batch_tst)
        tst_acc_metric.update_state(y_batch_tst, output)

    tst_acc = tst_acc_metric.result()

    # compute time required for each epoch
    end_time = time.time() - start_time

    print("test accuracy : %.4f \t time:  %.2f" % (  float(tst_acc), end_time))

In [ ]:
#TODO: show examples using the above datasets
# https://machinelearningmastery.com/diagnose-overfitting-underfitting-lstm-models/

# More resources:

* 3Blue1Brown Neural Network [video tutorials](https://www.youtube.com/playlist?list=PLZHQObOWTQDNU6R1_67000Dx_ZCJB-3pi) 
* Deep Learning Video Lectures by Prof. Andreas Maier [Winter 20/21](https://www.youtube.com/watch?v=SCFToE1vM2U&list=PLpOGQvPCDQzvJEPFUQ3mJz72GJ95jyZTh)
* Some of the code in this notebook is taken from [here](https://www.tensorflow.org/guide/keras/writing_a_training_loop_from_scratch)
* Calculating number of parameters in [CNN](https://towardsdatascience.com/understanding-and-calculating-the-number-of-parameters-in-convolution-neural-networks-cnns-fc88790d530d)
* Some of the code in this notebook is taken from [here](https://colab.research.google.com/github/tensorflow/docs/blob/master/site/en/tutorials/images/cnn.ipynb)
* https://nihcc.app.box.com/v/ChestXray-NIHCC
* https://www.tensorflow.org/datasets/catalog/patch_camelyon
* https://danielnouri.org/notes/2014/12/17/using-convolutional-neural-nets-to-detect-facial-keypoints-tutorial/
* https://elix-tech.github.io/ja/2016/06/02/kaggle-facial-keypoints-ja.html
* https://fairyonice.github.io/achieving-top-23-in-kaggles-facial-keypoints-detection-with-keras-tensorflow.html
* https://tensorflow-object-detection-api-tutorial.readthedocs.io/en/latest/training.html
* https://colab.research.google.com/github/tensorflow/hub/blob/master/examples/colab/object_detection.ipynb
* https://github.com/nicknochnack/TFODCourse
* https://www.analyticsvidhya.com/blog/2018/10/a-step-by-step-introduction-to-the-basic-object-detection-algorithms-part-1/
* https://github.com/enggen/Deep-Learning-Coursera
* https://github.com/prateeshreddy/Deep-Learning-Coursera
* https://github.com/JudasDie/deeplearning.ai
